## Lark parser

* https://github.com/lark-parser/lark
* https://lark-parser.readthedocs.io/en/latest/


### Simple grammar and transformer

In [ ]:
from lark import Lark
from pprint import pprint

l = Lark('''
start: reg+
reg: "reg" WORD "{" field* "}"
field: "field" WORD ";"

%import common.WORD
%import common.WS
%ignore WS
''')

data = '''reg Eenie { field eey; field eez; }
reg Meenie { field meh; }'''

top = l.parse(data)

print(top.pretty())
print('top.children[0].children =', top.children[0].children)
print('top.data =', top.data)

In [ ]:
from lark import Transformer

class MyTransformer(Transformer):
    def field(self, args):
        pprint(args)
        return str(args[0])
    def reg(self, args):
        pprint(args)
        return (str(args[0]), args[1:])

MyTransformer().transform(top).children

### More complex grammar, transformer and visitor (topdown)

In [ ]:
l2 = Lark('''
start: block+
block: "block" WORD "{" block_item* "}"
block_item: bytes | registers | endianness
bytes: "bytes" INT ";"
!endianness: "endian" ("little" | "big") ";"
registers: "register" WORD ";"

%import common.INT
%import common.WORD
%import common.WS

%ignore WS
''')
data2 = '''
block kezlok {
  bytes 2;
  register a;
  register n;
  endian little;
  register h;
}
block klancyk { }
'''
top2 = l2.parse(data2)
print(top2.pretty())

In [ ]:
from lark import Discard

class L2Transformer(Transformer):
    regs_a = []

    INT = int
    WORD = str
    def start(self, args):
        print("st ", args)
        return args
    def block(self, args):
        print("blk", args, args[0])
        rv = {'name':str(args[0]), 'regs':self.regs_a}
        self.regs_a = []
        return rv
    # If not defined, will not inline args, but return tree
    # def block_item(self, args):
    #     print("bli", args)
    #     return args
    def registers(self, args):
        print("reg", args, args[0])
        self.regs_a.append(str(args[0]))
        return args
    def bytes(self, args):
        print("byt", args)
        return args
    def endianness(self, args):
        print("end", args)
        return args[1]
        #return Discard
    # To skip rules, you can also define them like: "bytes: (...) -> skip"
    # def skip(self, args):
    #     return None

trx = L2Transformer().transform(top2)
trx
#trx.children[0]['regs'][0]

In [ ]:
from lark import Visitor

class L2visitor(Visitor):
    def __init__(self):
        self.blocks = []
    def block(self, tree):
        #print("tree: ", tree)
        self.blocks.append({'name':str(tree.children[0]), 'regs':[]})
    def registers(self, tree):
        #print("regs: ", tree)
        self.blocks[-1]['regs'].append(str(tree.children[0]))


vi2 = L2visitor()
vi2.visit_topdown(top2)
vi2.blocks